### Eluvio DS Challenge Option 1
#### Created by Chun Yin KONG

In [2]:
## Importing modules
import pandas as pd
import numpy as np

In [3]:
## reading data
data = pd.read_csv("Eluvio_DS_Challenge.csv")

### Understanding the data set

In [4]:

data.head()

,time_created,date_created,up_votes,down_votes,title,over_18,author,category
0,1201232046,2008-01-25,3,0,Scores killed in Pakistan clashes,False,polar,worldnews
1,1201232075,2008-01-25,2,0,Japan resumes refuelling mission,False,polar,worldnews
2,1201232523,2008-01-25,3,0,US presses Egypt on Gaza border,False,polar,worldnews
3,1201233290,2008-01-25,1,0,Jump-start economy: Give health care to all,False,fadi420,worldnews
4,1201274720,2008-01-25,4,0,Council of Europe bashes EU&UN terror blacklist,False,mhermans,worldnews


In [5]:
data.shape

(509236, 8)

In [7]:
data.isnull().any()

time_created    False
date_created    False
up_votes        False
down_votes      False
title           False
over_18         False
author          False
category        False
dtype: bool

In [27]:
col_list = list(data)
drop_col_list = []
for col_names in col_list:
    if len(data[str(col_names)].unique()) - 1 == 0:
        print(str(col_names), ": all duplicated values")
        drop_col_list.append(str(col_names))
    else:
        print(str(col_names), ": has unique values")

time_created : has unique values
date_created : has unique values
up_votes : has unique values
down_votes : all duplicated values
title : has unique values
over_18 : has unique values
author : has unique values
category : all duplicated values
['down_votes', 'category']


In [28]:
data = data.drop(drop_col_list, axis=1)

In [29]:
# after dropping all duplicated value's column
data.head()

,time_created,date_created,up_votes,title,over_18,author
0,1201232046,2008-01-25,3,Scores killed in Pakistan clashes,False,polar
1,1201232075,2008-01-25,2,Japan resumes refuelling mission,False,polar
2,1201232523,2008-01-25,3,US presses Egypt on Gaza border,False,polar
3,1201233290,2008-01-25,1,Jump-start economy: Give health care to all,False,fadi420
4,1201274720,2008-01-25,4,Council of Europe bashes EU&UN terror blacklist,False,mhermans
